<a href="https://colab.research.google.com/github/Manishtyy/sentiment/blob/main/Main_Copy_of_app_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installers

In [86]:
pip install google-play-scraper

In [87]:
from google_play_scraper import app, Sort, reviews_all
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json, os, uuid

Google and App Store reviews

In [88]:
# Ask the user to enter the package name and app ID
google_package_name = input("Enter the Android_app package name (e.g., com.digilocker.android): ")
apple_package_name = input("Enter the IOS_App's package name (e.g., digilocker):")
app_id = input("Enter the app ID (e.g., 1320618078): ")

Enter the Android_app package name (e.g., com.digilocker.android): com.digilocker.android
Enter the IOS_App's package name (e.g., digilocker):digilocker
Enter the app ID (e.g., 1320618078): 1320618078


In [89]:
# from numpy.ma import count
# g_reviews = reviews_all(
#         'flipboard.app',
#         sleep_milliseconds=0, # defaults to 0
#         lang='en', # defaults to 'en'
#         country='us', # defaults to 'us'
#         sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
#     )
# # Limit the number of Google Play Store reviews to 1000
# g_reviews = g_reviews[:1000]
# a_reviews = AppStore('us', 'flipboard-the-social-magazine', '358801284')
# a_reviews.review(limit=1000)

In [ ]:
import asyncio
from threading import Thread
from google_play_scraper import Sort, reviews_all
from app_store_scraper import AppStore

def scrape_reviews():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(async_scrape_reviews())

async def async_scrape_reviews():
    # Your scraping code for Google Play Store here
    g_reviews = reviews_all(google_package_name, sleep_milliseconds=0, lang='en', country='us', sort=Sort.NEWEST)
    g_reviews = g_reviews[:1000]

    # Your scraping code for Apple App Store here
    a_reviews = AppStore('us', apple_package_name, app_id)
    await a_reviews.review(limit=1000)

# Start the scraping in a new thread
scraping_thread = Thread(target=scrape_reviews)
scraping_thread.start()
scraping_thread.join()



Rename column names for google reviews


In [ ]:
g_df = pd.DataFrame(np.array(g_reviews),columns=['review'])
g_df2 = g_df.join(pd.DataFrame(g_df.pop('review').tolist()))

g_df2.drop(columns={'userImage', 'reviewCreatedVersion', 'at', 'thumbsUpCount', 'replyContent', 'userName' , 'repliedAt'},inplace = True)
g_df2.rename(columns= {'score': 'rating', 'reviewId': 'review_id', 'content': 'review_description'},inplace = True)
g_df2.insert(loc=0, column='source', value='Google Play')
g_df2.insert(loc=4, column='review_title', value=None)
g_df2['laguage_code'] = 'en'
g_df2['country_code'] = 'us'

Rename column names for App store reviews

In [ ]:
# a_df = pd.DataFrame(np.array(a_reviews.reviews),columns=['review'])
# a_df2 = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))

# a_df2.drop(columns={'isEdited', 'date', 'developerResponse' , 'userName'},inplace = True)
# a_df2.insert(loc=0, column='source', value='App Store')
# a_df2['app_version'] = None
# a_df2['laguage_code'] = 'en'
# a_df2['country_code'] = 'us'
# a_df2.insert(loc=1, column='review_id', value=[uuid.uuid4() for _ in range(len(a_df2.index))])
# a_df2.rename(columns= {'review': 'review_description','title': 'review_title'},inplace = True)
# a_df2 = a_df2.where(pd.notnull(a_df2), None)



In [ ]:
import pandas as pd
import numpy as np
import uuid

# Create DataFrame
a_df = pd.DataFrame(np.array(a_reviews.reviews), columns=['review'])
a_df.head()
a_df2 = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))

# Check if columns exist before attempting to drop
columns_to_drop = ['isEdited', 'developerResponse', 'date', 'userName']
for col in columns_to_drop:
    if col in a_df2.columns:
        a_df2.drop(columns=col, inplace=True)
    else:
        print(f"Column '{col}' not found in DataFrame.")

# Insert new columns
a_df2.insert(loc=0, column='source', value='App Store')
a_df2['app_version'] = None
a_df2['language_code'] = 'en'
a_df2['country_code'] = 'us'
a_df2.insert(loc=1, column='review_id', value=[uuid.uuid4() for _ in range(len(a_df2.index))])

# Rename columns
a_df2.rename(columns={'review': 'review_description', 'title': 'review_title'}, inplace=True)

# Replace NaN values with None
a_df2 = a_df2.where(pd.notnull(a_df2), None)
a_df2.head()

Result


In [ ]:


# result = pd.concat([g_df2,a_df2])
# result

In [ ]:
from google.colab import files

In [ ]:
output_file= 'App_reviews.csv'
a_df2.to_csv('App_reviews.csv', index=False)
g_df2.to_csv('App_reviews.csv', mode='a', header=False,  index=False)
files.download(output_file)

In [ ]:
from google.colab import files

In [ ]:
import pandas as pd
from textblob import TextBlob

# Load the CSV data into a DataFrame
df = pd.read_csv('App_reviews.csv')


# Define a function to perform sentiment analysis
def analyze_sentiment(text):
    analysis = TextBlob(str(text))
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis to a specific column (e.g., 'comments') and create a new column for sentiment
df['sentiment'] = df['review_description'].apply(analyze_sentiment)

# Save the updated DataFrame to a new CSV file
df.to_csv('sentiment_analysis_result.csv', index=False)
#files.download('sentiment_analysis_result.csv')

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Load the CSV data into a DataFrame
df = pd.read_csv('sentiment_analysis_result.csv')

# Preprocessing and tokenization
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = nltk.word_tokenize(str(text).lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)

df['cleaned_reviews'] = df['review_description'].apply(preprocess)

# CountVectorizer Vectorization
count_vectorizer = CountVectorizer(max_features=100)  # You can adjust the number of features (keywords)
count_matrix = count_vectorizer.fit_transform(df['cleaned_reviews'])

# Calculate keyword scores
feature_names = count_vectorizer.get_feature_names_out()
keyword_frequencies = count_matrix.sum(axis=0).tolist()[0]

# Create a dictionary of keywords and their scores
keywords_dict = {feature_names[i]: score for i, score in enumerate(keyword_frequencies)}

# Get the top N keywords
top_keywords = Counter(keywords_dict).most_common(50)  # Get the top 10 keywords

# Print and save top keywords and their scores to a CSV file
keyword_data = pd.DataFrame(top_keywords, columns=['Keyword', 'frequency'])
keyword_data.to_csv('keyword_analysis_result.csv' , index=False)
#download file to csv
# files.download('sentiment_analysis_result.csv')
# files.download('keyword_analysis_result.csv')

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Load the CSV data into a DataFrame
df = pd.read_csv('sentiment_analysis_result.csv')

# Preprocessing and tokenization
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = nltk.word_tokenize(str(text).lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)

df['cleaned_reviews'] = df['review_description'].apply(preprocess)

# Combine all cleaned reviews into a single text
all_text = ' '.join(df['cleaned_reviews'])

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

# Visualize the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Save the word cloud as an image
wordcloud.to_file('wordcloud.png')

# Optionally, save the word cloud data back to an existing CSV file
df.to_csv('reviews_with_wordcloud.csv', index=False)
#files.download('reviews_with_wordcloud.csv')
files.download('sentiment_analysis_result.csv')
files.download('keyword_analysis_result.csv')
files.download('wordcloud.png')